### Requirements

**Gantt chart with projects and project-specific tasks; created through a JSON file.**

Task attributes:
- start and end dates
- parent-project 
- description
- milestones
- status

Visuals should include:
- color-coding according to yearly quartile
- month and quarter axis frequency 
- task description at the bottom 
- milestones



----
#### Input

In [1]:
projects = {
    '1' : {
        '1': {
            'start': '2024-05-01',
            'end': '2024-06-15',
            'description': 'Systematic review on methodologies for seizure forecasting: contact Epilepsia editor, redo forest plot, rewrite document, appendix tables, and submit with B-on agreement.',
            'status': 'I',
        },
        '2': {
            'start': '2024-05-01',
            'end': '2024-06-30',
            'description': 'Bioentrepreneurship project.',
            'status': 'I',
        },
        '3': {
            'start': '2024-05-01',
            'end': '2024-06-15',
            'description': 'Systematic review on methodologies for seizure forecasting: contact Epilepsia editor, redo forest plot, rewrite document, appendix tables, and submit with B-on agreement.',
            'status': 'I',
        },
        '4': {
            'start': '2024-07-01',
            'end': '2024-07-15',
            'description': 'Systematic review on methodologies for seizure forecasting: contact Epilepsia editor, redo forest plot, rewrite document, appendix tables, and submit with B-on agreement.',
            'status': 'I',
        },
    },   
    '2' : {
        '1': {
            'start': '2024-05-01',
            'end': '2024-06-15',
            'description': 'Systematic review on methodologies for seizure forecasting: contact Epilepsia editor, redo forest plot, rewrite document, appendix tables, and submit with B-on agreement.',
            'status': 'I',
        },
        '2': {
            'start': '2024-05-01',
            'end': '2024-06-30',
            'description': 'Bioentrepreneurship project.',
            'status': 'I',
        },
        '3': {
            'start': '2024-05-01',
            'end': '2024-06-15',
            'description': 'Systematic review on methodologies for seizure forecasting: contact Epilepsia editor, redo forest plot, rewrite document, appendix tables, and submit with B-on agreement.',
            'status': 'I',
        },
        '4': {
            'start': '2024-07-01',
            'end': '2024-07-15',
            'description': 'Systematic review on methodologies for seizure forecasting: contact Epilepsia editor, redo forest plot, rewrite document, appendix tables, and submit with B-on agreement.',
            'status': 'I',
        },
        '5': {
            'start': '2024-10-01',
            'end': '2024-10-15',
            'description': 'Systematic review on methodologies for seizure forecasting: contact Epilepsia editor, redo forest plot, rewrite document, appendix tables, and submit with B-on agreement.',
            'status': 'I',
        },
    }   
}

In [2]:
color_palette = ['#5ADBFF', '#4EBDDC', '#5D9BCF', '#3D6997', '#675F94']

----
#### We need to transform the json into something like:

| task | projects | start | end | status | milestone | description |
| ---- | -------- | ----- | --- | ------ | --------- | ----------- | 
| task 1.1 | project 1 | 2024-05-01 | 2024-06-15 | I | ? | ? |

In [3]:
from gantt_timeline_aux import GanttTimeline

gantt_timeline = GanttTimeline(projects, color_palette)
gantt_timeline.df

,task,project,start,end,description,status,quarter,first,last,quarter_color
0,Project 1,None,None,None,None,None,None,2024-05-01,2024-10-15,NaN
1,Task 1.1,Project 1,2024-05-01,2024-06-15,Systematic review on methodologies for seizure...,I,2,2024-05-01,2024-10-15,#5ADBFF
2,Task 1.2,Project 1,2024-05-01,2024-06-30,Bioentrepreneurship project.,I,2,2024-05-01,2024-10-15,#5ADBFF
3,Task 1.3,Project 1,2024-05-01,2024-06-15,Systematic review on methodologies for seizure...,I,2,2024-05-01,2024-10-15,#5ADBFF
4,Task 1.4,Project 1,2024-07-01,2024-07-15,Systematic review on methodologies for seizure...,I,3,2024-05-01,2024-10-15,#4EBDDC
5,Project 2,None,None,None,None,None,None,2024-05-01,2024-10-15,NaN
6,Task 2.1,Project 2,2024-05-01,2024-06-15,Systematic review on methodologies for seizure...,I,2,2024-05-01,2024-10-15,#5ADBFF
7,Task 2.2,Project 2,2024-05-01,2024-06-30,Bioentrepreneurship project.,I,2,2024-05-01,2024-10-15,#5ADBFF
8,Task 2.3,Project 2,2024-05-01,2024-06-15,Systematic review on methodologies for seizure...,I,2,2024-05-01,2024-10-15,#5ADBFF
9,Task 2.4,Project 2,2024-07-01,2024-07-15,Systematic review on methodologies for seizure...,I,3,2024-05-01,2024-10-15,#4EBDDC


In [4]:
fig = gantt_timeline.get_fig_timeline(quarters=True, months=True)
fig.show()
